<a href="https://colab.research.google.com/github/lucabem/CompeticionMineria/blob/main/Presentacion_fine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers.normalization import BatchNormalization


In [ ]:
batch_size = 68
base_model = tf.keras.applications.ResNet50(input_shape = (224, 224, 3), 
                                            classes     = 25,
                                            include_top  = False)

image_batch, label_batch = next(iter(train_data_gen))
feature_batch = base_model(image_batch)
base_model.trainable = False

dense_layer_2048 = Dense(2048, activation='relu')
dense_layer_512  = Dense(512,  activation='relu')
dense_layer_1024 = Dense(1024, activation='relu')

conv_1024 = Conv2D(1024, (3,3), activation='relu')
conv_512  = Conv2D(512, (3,3), activation='relu')

flattern  = Flatten()
batch_normal = BatchNormalization()
dropout   = Dropout(0.25)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(25, activation='softmax')

model_22 = tf.keras.Sequential([
  base_model,
  conv_1024,
  conv_512,
  flattern,
  dense_layer_2048,
  dropout,
  dense_layer_512,
  dropout,
  dense_layer_1024,
  prediction_layer
])

In [ ]:
opt = tf.keras.optimizers.Adagrad()

callback = tf.keras.callbacks.EarlyStopping(monitor='f1_score', patience=5)

model_22.compile(loss="categorical_crossentropy",
              optimizer=opt,
	            metrics=[ 'accuracy', tfa.metrics.F1Score(num_classes=25, average='weighted')],
              )

In [ ]:
tf.random.set_seed(2021)

model_history_22 = model_22.fit(  
    train_data_gen,
    epochs=50,
    validation_data=valid_data_gen,
    validation_steps= valid_data_gen.n // batch_size)

Epoch 1/50
63/63 [==============================] - 508s 8s/step - loss: 2.9746 - accuracy: 0.1661 - f1_score: 0.1462 - val_loss: 1.5037 - val_accuracy: 0.6295 - val_f1_score: 0.5928
Epoch 2/50
63/63 [==============================] - 499s 8s/step - loss: 1.6180 - accuracy: 0.5325 - f1_score: 0.5221 - val_loss: 0.9249 - val_accuracy: 0.7321 - val_f1_score: 0.7271
Epoch 3/50
63/63 [==============================] - 497s 8s/step - loss: 1.0793 - accuracy: 0.6767 - f1_score: 0.6740 - val_loss: 0.9085 - val_accuracy: 0.7366 - val_f1_score: 0.7339
Epoch 4/50
63/63 [==============================] - 498s 8s/step - loss: 0.8001 - accuracy: 0.7627 - f1_score: 0.7626 - val_loss: 0.6996 - val_accuracy: 0.7991 - val_f1_score: 0.7967
Epoch 5/50
63/63 [==============================] - 496s 8s/step - loss: 0.7014 - accuracy: 0.7704 - f1_score: 0.7700 - val_loss: 0.5700 - val_accuracy: 0.8482 - val_f1_score: 0.8477
Epoch 6/50
63/63 [==============================] - 494s 8s/step - loss: 0.5703 - acc

In [ ]:
path_to_model = "modelo_mejorado2_2048_512_1024_256_128_52/"
model = tf.keras.models.load_model(path_to_model, compile=True, custom_objects={'f1':tfa.metrics.F1Score(num_classes=25, average='weighted')})

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback  

checkpointer = ModelCheckpoint("resnet50" +
                               "_ms_transfer_final." +
                               "{epoch:02d}-{val_f1_score:.3f}" +
                               ".hdf5",
                               monitor='val_f1_score',
                               verbose=1,
                               save_best_only=True,
                               mode='max')

earlystopper = EarlyStopping(monitor='val_f1_score',
                             patience=10,
                             mode='max')

In [ ]:
opt = tf.keras.optimizers.Adagrad()
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
	            metrics=[ 'accuracy', tfa.metrics.F1Score(num_classes=25, average='weighted')],
              )

In [ ]:
tf.random.set_seed(2021)
batch_size = 68
model_history = model.fit(  
    train_data_gen,
    epochs=70,
    initial_epoch = 50,
    validation_data=valid_data_gen,
    validation_steps= valid_data_gen.n // batch_size,
    callbacks=[checkpointer, earlystopper])

Epoch 51/70
63/63 [==============================] - 1790s 28s/step - loss: 1.4564 - accuracy: 0.5659 - f1_score: 0.5622 - val_loss: 2.4387 - val_accuracy: 0.5268 - val_f1_score: 0.4792

Epoch 00051: val_f1_score improved from -inf to 0.47919, saving model to resnet50_ms_transfer_final.51-0.479.hdf5
Epoch 52/70
63/63 [==============================] - 1804s 29s/step - loss: 0.4593 - accuracy: 0.8588 - f1_score: 0.8588 - val_loss: 1.7566 - val_accuracy: 0.5938 - val_f1_score: 0.5827

Epoch 00052: val_f1_score improved from 0.47919 to 0.58273, saving model to resnet50_ms_transfer_final.52-0.583.hdf5
Epoch 53/70
63/63 [==============================] - 1789s 28s/step - loss: 0.2313 - accuracy: 0.9283 - f1_score: 0.9281 - val_loss: 1.7505 - val_accuracy: 0.6384 - val_f1_score: 0.6391

Epoch 00053: val_f1_score improved from 0.58273 to 0.63907, saving model to resnet50_ms_transfer_final.53-0.639.hdf5
Epoch 54/70
63/63 [==============================] - 1814s 29s/step - loss: 0.1858 - accura

+ épocas (angeles)

In [ ]:
batch_size = 32
img_height = 224
img_width  = 224
test_path  = pathlib.Path('/root/.keras/datasets/Test')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.models import load_model
model_cargado = load_model('resnet50_ms_transfer_final.58-0.968.hdf5')

In [ ]:
for layer in model_cargado.layers:
   layer.trainable = True


for layer in model_cargado.get_layer('resnet50').layers[0:165]:
   layer.trainable = False

model_cargado.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 1024)        18875392  
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 512)         4719104   
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2048)              9439232   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)              

In [ ]:
model_history = model_cargado.fit(  
    train_data_gen,
    epochs=10,
    validation_data=valid_data_gen,
    validation_steps= valid_data_gen.n // batch_size)

Epoch 1/10
 6/63 [=>............................] - ETA: 24:09 - loss: 0.0389 - accuracy: 0.9896 - f1_score: 0.9896

+ epocas (luis)